# Inference

This notebook contains code that can be used to run inferences of models. The inference results are stored in various file formats and later they are analyzed in other notebooks.

## Machine translation

In [ ]:

def prepare_enabled_translator(translator_class, target_language, enabled=True):
    if translator_class == AmazonTranslate:
        return AmazonTranslate(
            target_language=target_language,
            enable_api=enabled,
        ).load()

    if translator_class == DeepL:
        return DeepL(
            target_language=target_language,
            enable_api=enabled,
            server_url='https://api.deepl.com/',
        ).load()

    if translator_class == GoogleTranslate:
        return GoogleTranslate(
            target_language=target_language,
            enable_api=enabled,
        ).load()

    if translator_class == NLLB:
        return NLLB(
            target_language=target_language,
            device='cuda:0',
            enable_inference=enabled,
        ).load()


data = list()
for translator_class in supported_languages.keys():
    print(translator_class)
    for language in supported_languages[translator_class]:
        print(language)
        translator = prepare_enabled_translator(translator_class, language, enabled=False)
        preds = predictions(translator_class, language, lazy=True)
        buf = []
        for sen, ste, pred in zip(sentences, stereotypes, preds):

            gender = None

            if pred == 'male':
                if translator_class == DeepL and language == 'cs':
                    prompt = f'She said "{sen}"'
                else:
                    prompt = f'She said: "{sen}"'
                # buf.append(prompt)
                
            if pred == 'female':
                if translator_class == DeepL and language == 'cs':
                    prompt = f'He said "{sen}"'
                else:
                    prompt = f'He said: "{sen}"'
                # buf.append(prompt)

            if pred:
                translation = translator.translate([prompt])[prompt]
                try:
                    extracted = extract_sentence(sen, translation)
                except:
                    print('Extraction failed:', translation)
                if extracted:
                    original = translator.translate([sen])[sen]
                    words_o, words_e = original.split(), extracted.split()
                    if len(words_o) == len(words_e) and sum(wo != we for wo, we in zip(words_o, words_e)) == 1:
                        if pred == 'male':
                            male, female = original, extracted
                        if pred == 'female':
                            male, female = extracted, original
                        data.append((
                            translator_class.__name__,
                            language,
                            sen,
                            ste,
                            male,
                            female,
                        ))                

            # if len(buf) == 250 or sen == sentences[-1]:
            #     translation = translator.translate(buf, save=True)
            #     for _from, _to in translation.items():
            #         print(_from)
            #         print(_to)
            #         print(unidecode(_to))
            #         print()
            #         break
            #     buf = []
        del translator            

In [ ]:
from unidecode import unidecode
from parser import Parser

import logging
logging.getLogger('deepl').setLevel(logging.WARNING)

for translator_class in translator_classes:
    for target_language in supported_languages[translator_class]:
        translator = translator_class(
            # dir_path='./cache/translations/nllb_3b',
            target_language=target_language,
            # enable_api=True,
            # server_url='https://api.deepl.com/',
        ).load()
        parser = Parser(language=target_language).load_model()
        for x in range(len(sentences)//100 + 1):   
            translations = translator.translate(sentences[x*100: x*100 + 100], save=True)  # Will be saved in translator dir
            parser.parse(list(translations.values()))  # Will be saved in parser dir


## English MLM

In [ ]:
templates = [
    (lambda s: f'He said: "{s}"', lambda s: f'She said: "{s}"'),
    (lambda s: f'The man said: "{s}"', lambda s: f'The woman said: "{s}"'),
    (lambda s: f'"{s}", he said.', lambda s: f'"{s}", she said.'),
    (lambda s: f'"{s}", the man said.', lambda s: f'"{s}", the woman said.'),
]

In [ ]:
def english_mlm_score(sample, templates, model, tokenizer, device):
    sen1, sen2 = templates[0](sample), templates[1](sample)
    a = calculate_logprob(sen1, sen2, tokenizer, model, device=device)
    b = calculate_logprob(sen2, sen1, tokenizer, model, device=device)
    return a - b

def make_predictions(model_handle, samples, templates, device):
    model, tokenizer = model_init(model_handle)
    return [
        english_mlm_score(sample, templates, model, tokenizer, device)
        for sample in tqdm(samples)
    ]

models = [
    'distilbert-base-uncased',
    'google/electra-large-generator',
    'google/electra-base-generator',
]

for model_handle in models:
    for i, template in enumerate(templates):
        preds = make_predictions(model_handle, sentences, template, 'cuda:0')
        import os
        os.makedirs('./data/predictions/english_mlm', exist_ok=True)
        with open(f'./cache/predictions/english_mlm/{model_handle.split("/")[-1]}_template-{i}.txt', 'w') as f:
            f.write('\n'.join(map(str, preds)))

In [ ]:
import requests

url = "https://huggingface.co/api/models"
payload = {"search": "google/multiberts"}
response = requests.get(url, params=payload)
handles = [
    hit['id']
    for hit in response.json()
]
handles

import os

for handle in handles[5:]:
    for t_id, template in enumerate(templates):
        preds = make_predictions(handle, sentences, template, 'cuda:0')
        dir_name = handle.split('/')[1]
        os.makedirs('./cache/predictions/multibert', exist_ok=True)
        with open(f'./cache/predictions/multibert/{dir_name}_template-{t_id}.txt', 'w') as f:
            f.write('\n'.join(map(str, preds)))

## Slavic MLM

In [ ]:
import re

patterns = [
    r'"(.+)"',
    r'„(.+)“',
    r'„(.+)”',
    r'“(.+)”',
    r'«(.+)»',
    r'»(.+)«',
    r'„(.+)"',
    r'"(.+)',
    r'„(.+)',
    r'„(.+)',
    r'»(.+)',
    r': (.+)',
    r'(.+)',
]

def extract_sentence(original, translation):
    if any((re_lst := re.findall(pattern, translation)) for pattern in patterns):
        extracted = re_lst[0]
        if original[-1] in '.?!' and extracted[-1] not in '.?!':
            extracted += o[-1]
        return extracted


In [ ]:
import pandas as pd

pd.set_option('display.max_rows', 500)

df = pd.DataFrame(data, columns=['translator', 'language', 'original', 'stereotype', 'male', 'female'])
df.to_csv('./data/gender_variants.csv', index=False)

In [ ]:
import random

male_buf, female_buf = list(), list()
for lang in ['be', 'ru', 'uk', 'hr', 'sl', 'sr', 'cs', 'pl', 'sk']:
    if translator_class in translator_classes:
        
        if lang not in supported_languages[translator_class]:
            continue

        translator = translator_class(target_language=lang).load()


        male_buf.extend([
            translator.dataframe.loc[sen]['to']
            for sen, pred in zip(sentences, predictions(translator_class, lang, lazy=True))
            if pred == 'male'
        ])
        female_buf.extend([
            translator.dataframe.loc[sen]['to']
            for sen, pred in zip(sentences, predictions(translator_class, lang, lazy=True))
            if pred == 'female'
        ])

for s in random.sample(male_buf, 50):
    print(s)
    print(unidecode(s))
print('=====')-
for s in random.sample(female_buf, 50):
    print(s)
    print(unidecode(s))
print('=====')
    
        


In [ ]:
def mlm_score(sample, model, tokenizer, device):
    sen1, sen2 = sample
    a = calculate_logprob(sen1, sen2, tokenizer, model, device=device)
    b = calculate_logprob(sen2, sen1, tokenizer, model, device=device)
    return a - b

def make_predictions(model_handle, samples, device):
    model, tokenizer = model_init(model_handle)
    return [
        mlm_score(sample, model, tokenizer, device)
        for sample in tqdm(samples)
    ]
    
for model_handle in models:
    preds = make_predictions(model_handle, list(zip(df.male, df.female)), 'cuda:0')
    import os
    os.makedirs('./cache/predictions/mmlm', exist_ok=True)
    with open(f'./cache/predictions/mmlm/{model_handle.split("/")[-1]}.txt', 'w') as f:
        f.write('\n'.join(map(str, preds)))